In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import os
from scipy.optimize import curve_fit

## Data preparation

### Load data

In [3]:
data_file_path = r'C:\Users\vaish\dsvv_atomionics\compiled_data\15795_chirp_up.csv'
data = pd.read_csv(data_file_path, usecols=
	[
		'chirp_rate', 'fraction',
		'CA+_mean', 'CA+_std_dev', 'CA+_percentile_0', 'CA+_percentile_10', 'CA+_percentile_20', 
		'CA+_percentile_30', 'CA+_percentile_40', 'CA+_percentile_50', 'CA+_percentile_60', 'CA+_percentile_70', 
		'CA+_percentile_80', 'CA+_percentile_90', 'CA+_percentile_100', 'CA-_mean', 'CA-_std_dev', 'CA-_percentile_0', 
		'CA-_percentile_10', 'CA-_percentile_20', 'CA-_percentile_30', 'CA-_percentile_40', 'CA-_percentile_50', 
		'CA-_percentile_60', 'CA-_percentile_70', 'CA-_percentile_80', 'CA-_percentile_90', 'CA-_percentile_100'
	])

# 80-20 train/test split
train_data_df = data.sample(frac=0.8, random_state=0)
test_data_df = data.drop(train_data_df.index)

print(train_data_df.shape)
# print(train_data.head())
print(test_data_df.shape)
# print(test_data.head())

(63, 28)
(16, 28)


### Constant parameters

In [4]:
# I'm not sure of the calculations here, copied from the original code
# constants for evaluation: keff, bigT, chirp_direction
speed_of_light_in_vacuum = 299792458
f1 = 384.2304844685e12 - 72.9113e6 - 2.56300597908911e9 + 61.625e6
f2 = f1 + 6.725e9 + 112.06936e6
k1 = 2*np.pi/(speed_of_light_in_vacuum/f1)
k2 = 2*np.pi/(speed_of_light_in_vacuum/f2)
Keff = k1+k2

bigT = 10e-3

chirp_direction = 'chirp_up'

## Custom Loss Function

In [5]:
def sine(alpha, contrast, acc_due_to_gravity, fringe_offset):
    phi = (Keff * acc_due_to_gravity - 2 * np.pi * alpha) * (bigT**2)
    return (-contrast * np.cos(phi) + fringe_offset)

In [6]:
def get_popt_for_best_fit(corrected_chirp_rates: np.ndarray, observed_fractions: np.ndarray):
	contrast = (np.max(observed_fractions) - np.min(observed_fractions)) / 2
	fringe_offset = np.mean(observed_fractions)
	g_guess = -9.78306 if chirp_direction == 'chirp_down' else 9.77997
	period = 780e-9/(2*bigT**2)

	sine_function_parameters = [contrast, g_guess, fringe_offset]
	bounds = ([0, g_guess - 0.501*period, -np.inf], [1, g_guess + 0.501*period, np.inf])
	popt, _ = curve_fit(sine, corrected_chirp_rates, observed_fractions, p0=sine_function_parameters, maxfev=1000000, bounds=bounds)
	return popt

In [7]:
def calculate_mse_loss_with_popt(outputs: torch.TensorType, observed_fractions: torch.TensorType, *popt):
	print(popt)
	tensor0 = torch.tensor([Keff * popt[1] * bigT**2 for i in range(len(outputs))], dtype=torch.float32)
	tensor1 = outputs * bigT**2 * 2 * torch.pi
	phi_tensor = torch.cos(tensor0 - tensor1)
	predicted_fractions_tensor = phi_tensor * (-popt[0]) + popt[2]
	mse_loss = nn.MSELoss()
	loss = mse_loss(predicted_fractions_tensor, observed_fractions)
	return loss

## Training

### Neural Network

In [8]:
class TransformationNN(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(TransformationNN, self).__init__()
        # Define the layers of the network: 3 fully connected layers 27 -> 64 -> 64 -> 1
        self.fc1 = nn.Linear(input_dim, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, output_dim)
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [9]:
training_data_tensor = torch.tensor(train_data_df.drop(columns=['fraction']).values, dtype=torch.float32)
observed_fractions_tensor = torch.tensor(train_data_df['fraction'].values, dtype=torch.float32)
observed_fractions_ndarray = train_data_df['fraction'].to_numpy()

In [10]:
# Initialize the neural network
model = TransformationNN(input_dim=27, output_dim=1)
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [11]:

num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    
    # Forward pass: Compute predicted y by passing x to the model
    outputs = model(training_data_tensor)
    
    outputs_clone = outputs.clone()
    corrected_chirp_rates_ndarray = outputs_clone.detach().cpu().squeeze().numpy()
    
    popt = get_popt_for_best_fit(corrected_chirp_rates_ndarray, observed_fractions_ndarray)    
    loss = calculate_mse_loss_with_popt(outputs, observed_fractions_tensor, *popt)
    
    # Print the loss
    print(f'Epoch [{epoch + 1}/{num_epochs}], MSE: {loss.item():.4f}')

    # Zero gradients, perform a backward pass, and update the weights
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

(0.999999999431646, 9.780892474735499, 0.8465546138507163)
Epoch [1/10], MSE: 0.0066
(0.09975007430612874, 9.778016100000002, 0.48235996377213886)
Epoch [2/10], MSE: 0.0066
(0.42318518977417396, 9.778016100000002, 0.16540348302552968)
Epoch [3/10], MSE: 0.0068
(0.9999999999629244, 9.778564739878222, -0.41575890528631854)
Epoch [4/10], MSE: 0.0067
(0.270108543000652, 9.778990218460214, 0.2871578444321634)
Epoch [5/10], MSE: 0.0063
(0.9999999999999998, 9.779424068611982, -0.4222207641966522)
Epoch [6/10], MSE: 0.0066
(0.3206116740120039, 9.778016100000002, 0.7777279999690003)
Epoch [7/10], MSE: 0.0065
(0.9999999999999999, 9.781441402323617, 1.550032765684551)
Epoch [8/10], MSE: 0.0063
(0.9999999999999999, 9.781358541988086, 1.5506371729435613)
Epoch [9/10], MSE: 0.0063


c:\Users\vaish\AppData\Local\Programs\Python\Python38\lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([63])) that is different to the input size (torch.Size([63, 63])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


(0.016823958955162477, 9.77801610000016, 0.5610285537112808)
Epoch [10/10], MSE: 0.0063
